# Summary

*   This file loads and merges data from two databases for residents to report odor complaints: [SmellMyCity (SMC)](https://smellmycity.org/) and [SeeClickFix (SCF)](https://seeclickfix.com/portland_2). The data is from 2020 and 2021. 
* 83% of SMC data is from S Portland. SCF is primarily used by Portland residents, according to Maria Guerra and Luke Truman.
* Only the SMC data specifies zipcodes. We filtered the SMC data to select all zipcodes in Portland and S Portland. 83% of the odor complaints are from zip code 04106, or the area of S Portland immediately to the south of the Fore River. 99.5% of the data isi from the three zipcodes immediately to the north and south of the Fore River (04106, 04101, 04102).
*   Key columns are: Report date, Lat/Long of user, Smell description (user text), Smell value (for SMC dataset only)
*   Separate columns were added for Year/Month/Day/Hour/Minute/Seconds

## Tidying next steps
*   Should this notebook produce the final, tidied dataframe in a way that can be accessed from a different notebook?
*   SMC data includes reports from Portland and S Portland. Try to determine whether there are any duplicate reports.



# Portland and S Portland zip codes


*   Portland
  *   04101, 04102: Portland immediately north of Fore River
  *   04103: Portland, further north (to northern border of the city)
  *   04108: Portland, islands to the east (Peaks Island and Cushing Island)
  *   04124: Portland, west of airport 

*   South Portland
  *   04106, 04107: S Portland immediately south of Fore River


In [36]:
import pandas as pd

# load SMC data
url_all_zips = 'https://raw.githubusercontent.com/caro28/ds5110_project/main/smell_reports_all_zipcodes.csv'
df_smc = pd.read_csv(url_all_zips)
print(df_smc.shape)

df_smc.head()

(2612, 9)


,epoch time,date & time,smell value,skewed latitude,skewed longitude,zipcode,smell description,symptoms,additional comments
0,1558531873,05/22/2019 09:31:13 -04:00,3,43.6608,-70.2498,4101,Grainy / malt like - coming from St. John street,NaN,NaN
1,1558691615,05/24/2019 05:53:35 -04:00,3,43.6435,-70.2702,4102,NaN,NaN,NaN
2,1559178135,05/29/2019 21:02:15 -04:00,3,43.6466,-70.2770,4102,Asphalty,NaN,NaN
3,1559341934,05/31/2019 18:32:14 -04:00,1,43.6325,-70.2828,4106,NaN,NaN,NaN
4,1559387558,06/01/2019 07:12:38 -04:00,3,43.6343,-70.2825,4106,Oil fumes,Throat irritation,NaN


In [37]:
df_smc['zipcode'].unique()

array([4101, 4102, 4106, 4107, 4103, 4108, 4124])

In [38]:
df_smc.groupby('zipcode').count()

,epoch time,date & time,smell value,skewed latitude,skewed longitude,smell description,symptoms,additional comments
zipcode,,,,,,,,
4101,53,53,53,53,53,43,16,2
4102,376,376,376,376,376,254,122,11
4103,5,5,5,5,5,4,4,1
4106,2169,2169,2169,2169,2169,1748,791,49
4107,7,7,7,7,7,3,1,0
4108,1,1,1,1,1,0,0,0
4124,1,1,1,1,1,0,0,0


In [39]:
df_smc.isnull().sum()

epoch time                0
date & time               0
smell value               0
skewed latitude           0
skewed longitude          0
zipcode                   0
smell description       560
symptoms               1678
additional comments    2549
dtype: int64

In [40]:
# Chnaging the date and time column to datetime format
df_smc['date & time']=df_smc['date & time'].str[0:20]
df_smc['date & time']=pd.to_datetime(df_smc['date & time'])

# Creating separate columns for date and time
df_smc['date'] = [d.date() for d in df_smc['date & time']]
df_smc['time'] = [d.time() for d in df_smc['date & time']]

# Creating a date and hour column
df_smc['Date & time (hour rounded)'] = df_smc['date & time'].dt.strftime("%Y-%m-%d %H:00:00")

# renaming columns
df_smc.rename(columns={'skewed latitude':'Latitude', 'skewed longitude':'Longitude'}, inplace=True)

df_smc

,epoch time,date & time,smell value,Latitude,Longitude,zipcode,smell description,symptoms,additional comments,date,time,Date & time (hour rounded)
0,1558531873,2019-05-22 09:31:13,3,43.6608,-70.2498,4101,Grainy / malt like - coming from St. John street,NaN,NaN,2019-05-22,09:31:13,2019-05-22 09:00:00
1,1558691615,2019-05-24 05:53:35,3,43.6435,-70.2702,4102,NaN,NaN,NaN,2019-05-24,05:53:35,2019-05-24 05:00:00
2,1559178135,2019-05-29 21:02:15,3,43.6466,-70.2770,4102,Asphalty,NaN,NaN,2019-05-29,21:02:15,2019-05-29 21:00:00
3,1559341934,2019-05-31 18:32:14,1,43.6325,-70.2828,4106,NaN,NaN,NaN,2019-05-31,18:32:14,2019-05-31 18:00:00
4,1559387558,2019-06-01 07:12:38,3,43.6343,-70.2825,4106,Oil fumes,Throat irritation,NaN,2019-06-01,07:12:38,2019-06-01 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
2607,1626995456,2021-07-22 19:10:56,4,43.6518,-70.2736,4102,NaN,NaN,NaN,2021-07-22,19:10:56,2021-07-22 19:00:00
2608,1627002279,2021-07-22 21:04:39,5,43.6340,-70.2849,4106,Tank fumes,NaN,NaN,2021-07-22,21:04:39,2021-07-22 21:00:00
2609,1627020315,2021-07-23 02:05:15,5,43.6428,-70.2452,4106,Petroleum smell most nites at 2am!!,NaN,NaN,2021-07-23,02:05:15,2021-07-23 02:00:00
2610,1627043324,2021-07-23 08:28:44,3,43.6325,-70.2731,4106,Tar,NaN,NaN,2021-07-23,08:28:44,2021-07-23 08:00:00


In [41]:
# load SCF data
url = 'https://raw.githubusercontent.com/caro28/ds5110_project/main/SCF%20ODOR%20REPORT%201-1-20%20to%206-3-21.csv'
df_scf=pd.read_csv(url)
print(df_scf.shape)
df_scf.head()

(295, 12)


,Id,Report Source,Category,Created at local,Closed at local,Status,Address,Description,URL,Lat,Lng,Export tagged places
0,7181157,iPhone,Odor,01/07/2020 - 08:26AM,01/07/2020 - 09:20AM,Archived,315 Spring Street,Petroleum smell coming from south portland,https://crm.seeclickfix.com/#/organizations/61...,43.647740,-70.269455,City Council District 2
1,7181402,Android,Odor,01/07/2020 - 09:11AM,01/07/2020 - 09:20AM,Archived,25 Cushman St,usual petroleum,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2
2,7192000,Android,Odor,01/09/2020 - 07:14AM,01/09/2020 - 08:45AM,Archived,25 Cushman St,usual petroleum,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2
3,7206428,Android,Odor,01/13/2020 - 08:22AM,01/13/2020 - 09:09AM,Archived,25 Cushman St,worst yet,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2
4,7210067,Android,Odor,01/14/2020 - 08:24AM,01/14/2020 - 02:50PM,Archived,25 Cushman St,usual petroleum stink. Cushman and Reiche play...,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2


In [42]:
df_scf.isnull().sum()

Id                       0
Report Source            0
Category                 0
Created at local         0
Closed at local          2
Status                   0
Address                  0
Description             39
URL                      0
Lat                      0
Lng                      0
Export tagged places     0
dtype: int64

In [43]:
# Changing the date and time column to datetime format
df_scf['Created at local']=pd.to_datetime(df_scf['Created at local'])
df_scf['Closed at local']=pd.to_datetime(df_scf['Closed at local'])

# Creating separate columns for date and time
df_scf['date'] = [d.date() for d in df_scf['Created at local']]
df_scf['time'] = [d.time() for d in df_scf['Created at local']]

# Creating a date and hour column (using the 'Created at local' column)
df_scf['Date & time (hour rounded)'] = df_scf['Created at local'].dt.strftime("%Y-%m-%d %H:00:00")

# renaming columns
df_scf.rename(columns={'Description':'smell description', 'Lat':'Latitude', 'Lng':'Longitude'}, inplace=True)

df_scf

,Id,Report Source,Category,Created at local,Closed at local,Status,Address,smell description,URL,Latitude,Longitude,Export tagged places,date,time,Date & time (hour rounded)
0,7181157,iPhone,Odor,2020-01-07 08:26:00,2020-01-07 09:20:00,Archived,315 Spring Street,Petroleum smell coming from south portland,https://crm.seeclickfix.com/#/organizations/61...,43.647740,-70.269455,City Council District 2,2020-01-07,08:26:00,2020-01-07 08:00:00
1,7181402,Android,Odor,2020-01-07 09:11:00,2020-01-07 09:20:00,Archived,25 Cushman St,usual petroleum,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2,2020-01-07,09:11:00,2020-01-07 09:00:00
2,7192000,Android,Odor,2020-01-09 07:14:00,2020-01-09 08:45:00,Archived,25 Cushman St,usual petroleum,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2,2020-01-09,07:14:00,2020-01-09 07:00:00
3,7206428,Android,Odor,2020-01-13 08:22:00,2020-01-13 09:09:00,Archived,25 Cushman St,worst yet,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2,2020-01-13,08:22:00,2020-01-13 08:00:00
4,7210067,Android,Odor,2020-01-14 08:24:00,2020-01-14 14:50:00,Archived,25 Cushman St,usual petroleum stink. Cushman and Reiche play...,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2,2020-01-14,08:24:00,2020-01-14 08:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,9986497,Portal,Odor,2021-05-24 16:31:00,NaT,Archived,14 Lawn Ave,Very strong oil smell,https://crm.seeclickfix.com/#/organizations/61...,43.670415,-70.293159,City Council District 5,2021-05-24,16:31:00,2021-05-24 16:00:00
291,9989334,Web - Mobile,Odor,2021-05-25 02:31:00,2021-05-25 08:16:00,Archived,Carroll St & Western Prom,petro/chem smell in the air,https://crm.seeclickfix.com/#/organizations/61...,43.648216,-70.275764,City Council District 2,2021-05-25,02:31:00,2021-05-25 02:00:00
292,10033882,Android,Odor,2021-06-01 07:49:00,2021-06-02 15:03:00,Archived,331 Spring St,NaN,https://crm.seeclickfix.com/#/organizations/61...,43.647309,-70.269934,City Council District 2,2021-06-01,07:49:00,2021-06-01 07:00:00
293,10040039,iPhone,Odor,2021-06-01 20:29:00,2021-06-02 09:58:00,Archived,15 Vaughan St,Moderate odor,https://crm.seeclickfix.com/#/organizations/61...,43.644839,-70.272152,City Council District 2,2021-06-01,20:29:00,2021-06-01 20:00:00


In [44]:
# Merge two datasets
df_stinky = df_scf.append(df_smc, sort=False)
df_stinky

,Id,Report Source,Category,Created at local,Closed at local,Status,Address,smell description,URL,Latitude,Longitude,Export tagged places,date,time,Date & time (hour rounded),epoch time,date & time,smell value,zipcode,symptoms,additional comments
0,7181157.0,iPhone,Odor,2020-01-07 08:26:00,2020-01-07 09:20:00,Archived,315 Spring Street,Petroleum smell coming from south portland,https://crm.seeclickfix.com/#/organizations/61...,43.647740,-70.269455,City Council District 2,2020-01-07,08:26:00,2020-01-07 08:00:00,NaN,NaT,NaN,NaN,NaN,NaN
1,7181402.0,Android,Odor,2020-01-07 09:11:00,2020-01-07 09:20:00,Archived,25 Cushman St,usual petroleum,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2,2020-01-07,09:11:00,2020-01-07 09:00:00,NaN,NaT,NaN,NaN,NaN,NaN
2,7192000.0,Android,Odor,2020-01-09 07:14:00,2020-01-09 08:45:00,Archived,25 Cushman St,usual petroleum,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2,2020-01-09,07:14:00,2020-01-09 07:00:00,NaN,NaT,NaN,NaN,NaN,NaN
3,7206428.0,Android,Odor,2020-01-13 08:22:00,2020-01-13 09:09:00,Archived,25 Cushman St,worst yet,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2,2020-01-13,08:22:00,2020-01-13 08:00:00,NaN,NaT,NaN,NaN,NaN,NaN
4,7210067.0,Android,Odor,2020-01-14 08:24:00,2020-01-14 14:50:00,Archived,25 Cushman St,usual petroleum stink. Cushman and Reiche play...,https://crm.seeclickfix.com/#/organizations/61...,43.649448,-70.268626,City Council District 2,2020-01-14,08:24:00,2020-01-14 08:00:00,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,43.651800,-70.273600,NaN,2021-07-22,19:10:56,2021-07-22 19:00:00,1.626995e+09,2021-07-22 19:10:56,4.0,4102.0,NaN,NaN
2608,NaN,NaN,NaN,NaT,NaT,NaN,NaN,Tank fumes,NaN,43.634000,-70.284900,NaN,2021-07-22,21:04:39,2021-07-22 21:00:00,1.627002e+09,2021-07-22 21:04:39,5.0,4106.0,NaN,NaN
2609,NaN,NaN,NaN,NaT,NaT,NaN,NaN,Petroleum smell most nites at 2am!!,NaN,43.642800,-70.245200,NaN,2021-07-23,02:05:15,2021-07-23 02:00:00,1.627020e+09,2021-07-23 02:05:15,5.0,4106.0,NaN,NaN
2610,NaN,NaN,NaN,NaT,NaT,NaN,NaN,Tar,NaN,43.632500,-70.273100,NaN,2021-07-23,08:28:44,2021-07-23 08:00:00,1.627043e+09,2021-07-23 08:28:44,3.0,4106.0,NaN,NaN


In [45]:
# check that common columns were correctly merged
df_stinky.isnull().sum()

Id                            2612
Report Source                 2612
Category                      2612
Created at local              2612
Closed at local               2614
Status                        2612
Address                       2612
smell description              599
URL                           2612
Latitude                         0
Longitude                        0
Export tagged places          2612
date                             0
time                             0
Date & time (hour rounded)       0
epoch time                     295
date & time                    295
smell value                    295
zipcode                        295
symptoms                      1973
additional comments           2844
dtype: int64

In [46]:
# look into missing 'smell description' data to check there were no merge errors
print('SCF has {} null smell description rows'.format(df_scf['smell description'].isnull().sum()))
print('SMC has {} null smell description rows'.format(df_smc['smell description'].isnull().sum()))
print('Merged df has {} null smell description rows'.format(df_stinky['smell description'].isnull().sum()))

SCF has 39 null smell description rows
SMC has 560 null smell description rows
Merged df has 599 null smell description rows


In [47]:
# download tidied df_stinky
from google.colab import files
df_stinky.to_csv('df_stinky.csv') 
files.download('df_stinky.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>